In [1]:
# @title Preparation
!pip install -q keras-bert
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -o uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [2]:
# @title Environment
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

In [3]:
# @title Load Basic Model
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=False,
    seq_len=300,
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
# @title Build Custom Model
from tensorflow.python import keras
from keras_bert.optimizers import AdamWarmup

inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=2, activation='softmax')(dense)

model = keras.models.Model(inputs, outputs)
model.compile(
    AdamWarmup(decay_steps=100000, warmup_steps=1000),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [5]:
# @title Initialize Variables
import tensorflow as tf
import tensorflow.keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [6]:
# @title Convert to TPU Model
import tensorflow as tf
from keras_bert import get_custom_objects

tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
)

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.12.162.170:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 9447741190219693902)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17426934761049643520)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12836661232812106036)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU

In [7]:
# @title Download IMDB Data
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [8]:
# @title Convert Data to Array
import os
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer

tokenizer = Tokenizer(token_dict)


def load_data(path):
    global tokenizer
    indices, sentiments = [], []
    for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in tqdm(os.listdir(folder)):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()
            ids, segments = tokenizer.encode(text, max_len=300)
            indices.append(ids)
            sentiments.append(sentiment)
    items = list(zip(indices, sentiments))
    np.random.shuffle(items)
    indices, sentiments = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(sentiments)
  
  
  
train_path = os.path.join(os.path.dirname(dataset), 'aclImdb', 'train')
test_path = os.path.join(os.path.dirname(dataset), 'aclImdb', 'test')


train_x, train_y = load_data(train_path)
test_x, test_y = load_data(test_path)

100%|██████████| 12500/12500 [00:46<00:00, 270.47it/s]


In [9]:
# @title Fit

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    tpu_model.fit(train_x, train_y, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.4440 - sparse_categorical_accuracy: 0.7949
Epoch 2/10
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.4345 - sparse_categorical_accuracy: 0.8042
Epoch 3/10
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.4354 - sparse_categorical_accuracy: 0.8007
Epoch 4/10
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.4286 - sparse_categorical_accuracy: 0.8058
Epoch 5/10
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.4240 - sparse_categorical_accuracy: 0.8058
Epoch 6/10
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.4244 - sparse_categorical_accuracy: 0.8056
Epoch 7/10
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.4234 - sparse_categorical_accuracy: 0.8069
Epoch 8/10
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.4193 - sparse_categorical_accuracy:

In [10]:
# @title Predict

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    predicts = tpu_model.predict(test_x, verbose=True).argmax(axis=-1)

25000/25000 [==============================] - 37s 1ms/sample


In [11]:
# @title Accuracy

print(np.sum(test_y == predicts) / test_y.shape[0])

0.82036
